# ようこそ

ここでは、DMPに入力いただいた情報に基づき、研究活動の支援、およびデータ管理品質向上のためにデータガバナンス機能が提供するリサーチフローを作成します。<br>
リサーチフローはJupyer Notebookの形式で提供されます。<br>
リサーチフローの各タスクに書かれているコードはご自身に合わせて編集することが可能です。<br>
ただし、編集内容によってはコードが正常に動作しなくなる可能性がありますのでご留意ください。<br>

Jupyter Notebookの操作方法については、[こちら](https://support.rdm.nii.ac.jp/usermanual/60/#jp_operation)をご覧ください。新規タブで開きます。

# maDMPを実行する

## 1. DMPの情報からリサーチフローを作成する
このmaDMPには、作成いただいたDMPの情報の中で、リサーチフローの作成に必要な情報が既に埋め込まれています。  
以下のセルを**上から順に全て実行**してください。  

In [ ]:
# DMP情報
workflowIdentifier = '%v'
contentSize = '%v'
datasetStructure = '%v'
useDocker = '%v'

In [ ]:
# テンプレート取得
clone_url = 'https://github.com/NII-DG/workflow-template'+'.git'
workflow_path = '/home/jovyan/WORKFLOWS'
!git clone $clone_url $workflow_path
!rm -rf $workflow_path/.git

In [ ]:
import requests
from WORKFLOWS.except_class import query_err
from WORKFLOWS.utils import display_util
from WORKFLOWS.utils.param_json import param_json
from WORKFLOWS.utils.token import token

# WORKFLOWS/FLOW/param_files/params.json の　"siblings": "ginHttp"　"ginSsh":を更新する。
origin_urls = !git config remote.origin.url
url = origin_urls[0]
try:
    # update param json
    param_json.update_param_url(url)
    # delete build token(only private)
    token.del_build_token_by_remote_origin_url(url)
except requests.exceptions.RequestException:
    display_util.display_err("現在、通信不良が発生しています。")

In [ ]:
# dmp.jsonに"fields"プロパティがある想定
import os
import glob

path_flows = os.path.join('WORKFLOWS', 'FLOW')

templates = glob.glob(os.path.join(path_flows, '**'), recursive=True)

# 選択外の分野のセクション群を削除
for tmpl in templates:
    file = os.path.basename(tmpl)
    if not os.path.isdir(tmpl) and os.path.splitext(file)[1] == '.ipynb':
        if 'base_' not in file and workflowIdentifier not in file:
            os.remove(tmpl)

## 2. フロー図を準備する

次に、研究フローが俯瞰できる図を表示する準備をします。  
以下のセルを実行してください。  
実行完了後、表示されるリンクをクリックして「研究フロートップページ」に遷移してください。  

In [ ]:
import nbformat
from IPython.display import display, HTML, Javascript
from WORKFLOWS.utils import display_util

flow_path = 'WORKFLOWS/base_FLOW.ipynb'
rcos_binder_url = 'https://jupyter.cs.rcos.nii.ac.jp/'
flow_path_notebook = 'notebooks/WORKFLOWS/base_FLOW.ipynb'
# flow_path_lab = 'lab/tree/WORKFLOW/FLOW/base_FLOW.ipynb'

nb = nbformat.read(flow_path, 4)

title1 = """## フロー図を表示する

以下のセルを実行してフロー図を表示できます。  
フロー図が非表示の場合や既に実行したタスクを確認したい場合はセルを再実行してください。"""

code1 = """import os
from IPython.display import display, SVG, Javascript

nb_utils_path   = os.path.join(os.environ['HOME'], 'WORKFLOWS/FLOW/util/scripts/nb_utils.py')

basic_path      = os.path.join(os.environ['HOME'], 'WORKFLOWS/images')
basic_path_diag = os.path.join(basic_path, 'notebooks.diag')
basic_path_svg  = os.path.join(basic_path, 'notebooks.svg')

os.chdir(os.environ['HOME'])
!python3 $nb_utils_path $basic_path_diag

display(SVG(filename=basic_path_svg))
display(Javascript('IPython.notebook.save_checkpoint();'))"""

title2 = """フロー図の左側にあるノードはフローのフェーズとその流れを表しており、右側にあるノードは個々の支援するタスクを表しています。  
各タスクを実行するには、実行したいタスクのノードのリンクをクリックしてください。"""

title3 = """## GIN-forkへ戻る

以下のセルを実行し、表示されるボタンをクリックするとGIN-forkへアクセスできます。  
ボタンをクリックして404エラーが表示された場合は、リポジトリ名が更新されている可能性があるため以下のセルを再度実行して下さい。"""

code2 = """import os
import subprocess
from IPython.display import display, Javascript, clear_output, HTML
from utils import display_util

# リモートリポジトリのURLを最新化する
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
is_new_private = utils.update_repo_url()
os.chdir(os.environ['HOME'])

# GIN-forkへ遷移するボタンを表示する
clear_output()
display_util.display_info('以下のボタンをクリックしてGIN-forkへ遷移してください。新規タブで開きます。')
url = subprocess.getoutput('git config --get remote.origin.url')
display(HTML(f'<button onclick="window.open(\\'{url}\\')">GIN-forkに遷移する</button>'))

if not is_new_private['is_new']:
    display_util.display_warm("最新のURLでない可能性があります。戻れない場合は初期セットアップ後に再実行してください。")
display(Javascript('IPython.notebook.save_checkpoint();'))"""

# フロー図作成セルが無い場合のみbase_FLOW.ipynbにセルを追加する
if '## フロー図を作成する' in str(nb):
    display_util.display_warm('フロー図はすでに準備されています。以下のリンクをクリックしてください。')
else:
    nb["cells"] += [
        nbformat.v4.new_markdown_cell(title1),
        nbformat.v4.new_code_cell(code1),
        nbformat.v4.new_markdown_cell(title2),
        nbformat.v4.new_markdown_cell(title3),
        nbformat.v4.new_code_cell(code2)
    ]
    with open(flow_path, "w") as f:
        nbformat.write(nb, f)
    display_util.display_info('フロー図の準備が完了しました。以下のリンクをクリックしてください。')
display(HTML("<a href='./WORKFLOWS/base_FLOW.ipynb'>研究フロートップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))